<a href="https://colab.research.google.com/github/mori8/NLP-Pytorch-practice/blob/main/pytorch_lecture/Chapter_10_Basic_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, datasets

In [8]:
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/content/drive/MyDrive/NLP-Pytorch/data', train=True, download=True,
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])),
                   batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/content/drive/MyDrive/NLP-Pytorch/data', train=False, download=True,
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])),
                   batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/NLP-Pytorch/data/MNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/NLP-Pytorch/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/NLP-Pytorch/data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/NLP-Pytorch/data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/NLP-Pytorch/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/NLP-Pytorch/data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/NLP-Pytorch/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/NLP-Pytorch/data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [65]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  # 필터를 10개 쓰겠다는 것? 
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.mp = nn.MaxPool2d(2)

    self.fc = nn.Linear(320, 10)  # fully connected layer, 아무 값이나 집어넣고 돌리면 파이토치가 정확한 값 알려줌

  def forward(self, x):
    in_size = x.size(0)  # batch size
    x = F.relu(self.mp(self.conv1(x)))
    x = F.relu(self.mp(self.conv2(x)))
    x = x.view(in_size, -1)  # flatten the tensor
    x = self.fc(x)
    return F.log_softmax(x)

In [71]:
# Exercise 10-1
class Net_exercise(nn.Module):
  def __init__(self):
    super(Net_exercise, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=2)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=2)
    self.conv3 = nn.Conv2d(20, 30, kernel_size=2)
    self.mp = nn.MaxPool2d(2)

    self.fc1 = nn.Linear(120, 60)
    self.fc2 = nn.Linear(60, 10)

  def forward(self, x):
    in_size = x.size(0)  # batch size
    x = F.relu(self.mp(self.conv1(x)))
    x = F.relu(self.mp(self.conv2(x)))
    x = F.relu(self.mp(self.conv3(x)))
    x = x.view(in_size, -1)  # flatten the tensor
    x = self.fc1(x)
    x = self.fc2(x)
    return F.log_softmax(x)

In [67]:
model = Net()

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [68]:
def train(epoch):
  model.train()
  for batch_idx, data in enumerate(train_loader):
    data, target = map(Variable, data)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [69]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    test_loss += criterion(output, target).data.item()
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
  
  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [70]:
for epoch in range(1, 10):
  train(epoch)
  test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.334127
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.542086
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.347946
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.241538
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.137466
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.349783
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.070847
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.118421
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.076337
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.151033
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.179603
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.158665
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.083529
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.339557
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.267592
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.057270
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.120154
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.308799
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.099919
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.124468
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0015, Accuracy: 9737/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.223574
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.063681
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.021095
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.064414
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.053928
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.233602
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.273018
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.104778
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.025811
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.003782
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.051248
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.013270
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.049036
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.056720
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.082572
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.135417
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.119770
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.034390
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.044488
Train Epoch: 2 [12

In [72]:
model_ex = Net_exercise()

optimizer = optim.Adam(model_ex.parameters(), lr=0.01)

In [73]:
def train_ex(epoch):
  model_ex.train()
  for batch_idx, data in enumerate(train_loader):
    data, target = map(Variable, data)
    optimizer.zero_grad()
    output = model_ex(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [74]:
def test_ex():
  model_ex.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model_ex(data)
    test_loss += criterion(output, target).data.item()
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
  
  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [75]:
for epoch in range(1, 4):
  train_ex(epoch)
  test_ex()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.315691
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.338377
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.575534
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.474696
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.246570
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.288771
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.515927
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.291435
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.323849
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.554246
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.286933
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.203731
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.316054
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.147083
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.381608
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.151995
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.320977
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.285903
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.204847
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.177011
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0024, Accuracy: 9561/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.074140
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.272920
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.108415
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.172322
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.075756
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.047023
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.023546
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.193978
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.069704
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.198005
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.068564
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.078537
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.111160
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.027999
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.226113
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.129102
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.397899
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.188947
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.131082
Train Epoch: 2 [12